# transform_sk
This transformation script is for STN version.


## Part 1
Transform skeletons by translation, rotate, scaling, and half-resize.

Change input_dir & output_dir


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def resize_half_image(img, scale_factor=1.5, resize_upper=True):
    """
    對圖片的一半進行比例改變
    
    參數:
        img: 輸入圖片 (BGR格式)
        scale_factor: 縮放比例 (例如 1.5 表示拉伸1.5倍，0.7 表示壓縮到0.7倍)
        resize_upper: True 表示改變上半部分，False 表示改變下半部分
    
    返回:
        處理後的圖片
    """
    height, width = img.shape[:2]
    mid_point = height // 2
    
    if resize_upper:
        # 改變上半部分
        upper_half = img[0:mid_point, :]
        lower_half = img[mid_point:, :]
        
        # 縮放上半部分
        new_upper_height = int(upper_half.shape[0] * scale_factor)
        upper_half_resized = cv2.resize(upper_half, (width, new_upper_height), 
                                        interpolation=cv2.INTER_LINEAR)
        
        # 組合
        total_height = new_upper_height + lower_half.shape[0]
        result_img = np.zeros((total_height, width, 3), dtype=np.uint8)
        result_img[0:new_upper_height, :] = upper_half_resized
        result_img[new_upper_height:, :] = lower_half
        
    else:
        # 改變下半部分
        upper_half = img[0:mid_point, :]
        lower_half = img[mid_point:, :]
        
        # 縮放下半部分
        new_lower_height = int(lower_half.shape[0] * scale_factor)
        lower_half_resized = cv2.resize(lower_half, (width, new_lower_height), 
                                        interpolation=cv2.INTER_LINEAR)
        
        # 組合
        total_height = upper_half.shape[0] + new_lower_height
        result_img = np.zeros((total_height, width, 3), dtype=np.uint8)
        result_img[0:upper_half.shape[0], :] = upper_half
        result_img[upper_half.shape[0]:, :] = lower_half_resized
    
    return result_img

In [ ]:
def rotate_image(img, angle, scale=1.0):
    """
    旋轉圖片
    
    參數:
        img: 輸入圖片
        angle: 旋轉角度 (正值逆時針，負值順時針)
        scale: 縮放比例 (預設 1.0 不縮放)
    """
    height, width = img.shape[:2]
    center = (width // 2, height // 2)
    
    # 取得旋轉矩陣
    M = cv2.getRotationMatrix2D(center, angle, scale)
    
    # 執行仿射轉換
    rotated_img = cv2.warpAffine(img, M, (width, height))
    
    return rotated_img



In [ ]:
import random
import cv2
import numpy as np

def scale_image(
    img,
    scale_percent=None,          # 改成 None 表示「沒特別指定」
    mode=None,                   # mode = 0 or 1，用來決定隨機區間
    border_percent_when_upscale=10,
    pad_value=0,
):
    """
    縮放圖片並加上 padding。
    
    - 有給 scale_percent (>0)：直接用這個數字
    - 沒給 scale_percent：
        * mode = 0 → 在 [80, 99] 之間隨機
        * mode = 1 → 在 [101, 120] 之間隨機
    - scale_percent < 100：縮小後貼到「原圖大小」的畫布中(四周是 padding)
    - scale_percent > 100：放大後再貼到「比放大後更大一點」的畫布中(四周是 padding)
    """
    h, w = img.shape[:2]

    # ---------- 決定 scale_percent ----------
    if scale_percent is None or scale_percent <= 0:
        if mode == 0:
            scale_percent = random.randint(80, 99)
        elif mode == 1:
            scale_percent = random.randint(101, 120)
        else:
            # 沒給 mode 的 fallback：不縮放
            scale_percent = 100

    # 1. 計算縮放後大小
    new_w = int(w * scale_percent / 100)
    new_h = int(h * scale_percent / 100)

    # 避免 new_w 或 new_h 變成 0
    new_w = max(new_w, 1)
    new_h = max(new_h, 1)

    # 2. 先做縮放
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # ---------- 情況一：縮小或等大（≤100） ----------
    if scale_percent <= 100:
        # 建立跟原圖同大小的畫布
        if img.ndim == 2:  # 灰階
            canvas = np.full((h, w), pad_value, dtype=img.dtype)
        else:              # 彩色
            canvas = np.full((h, w, img.shape[2]), pad_value, dtype=img.dtype)

        # 置中貼上縮小後圖片
        x_offset = (w - new_w) // 2
        y_offset = (h - new_h) // 2
        canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
        return canvas

    # ---------- 情況二：放大 (>100) ----------
    # 放大時再加一圈 padding，避免放大後畫面剛好貼邊
    border_x = int(new_w * border_percent_when_upscale / 100)
    border_y = int(new_h * border_percent_when_upscale / 100)

    canvas_w = new_w + 2 * border_x
    canvas_h = new_h + 2 * border_y

    if img.ndim == 2:
        canvas = np.full((canvas_h, canvas_w), pad_value, dtype=img.dtype)
    else:
        canvas = np.full((canvas_h, canvas_w, img.shape[2]), pad_value, dtype=img.dtype)

    # 讓放大的圖置中
    x_offset = border_x
    y_offset = border_y
    canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

    return canvas


In [ ]:
import random

def scale_then_rotate_random(
    img,
    angle=None,        # 如果你想固定角度可以直接給，否則就用 mode 決定隨機範圍
    mode=None,         # 0: 正 30 度以內；1: -30 到 0 度；其他: -30 ~ +30
    max_angle=30,
    scale_percent=80
):
    """
    先縮小，再用隨機角度旋轉。

    - 如果 angle 有給值：直接用 angle
    - 如果 angle 沒給：
        * mode = 0 → 隨機在 [0, +max_angle]
        * mode = 1 → 隨機在 [-max_angle, 0]
        * 其他 / None → 隨機在 [-max_angle, +max_angle]
    """

    # 決定角度
    if angle is None:
        if mode == 0:
            angle = random.uniform(0, max_angle)
        elif mode == 1:
            angle = random.uniform(-max_angle, 0)
        else:
            angle = random.uniform(-max_angle, max_angle)

    # 先縮小
    scaled = scale_image(img, scale_percent=scale_percent)
    # 再旋轉
    rotated = rotate_image(scaled, angle=angle)
    return rotated


In [ ]:
import random
import numpy as np

def translate_image_wrap(img, shift_x=0, shift_y=0, mode=None):
    """
    平移圖片 (循環位移)
    - mode 控制隨機平移：
        mode = 0: shift_x 在 [-90, -60] 之間隨機（往左）
        mode = 1: shift_x 在 [60, 90] 之間隨機（往右）
        mode = 2: shift_y 在 [-90, -60] 之間隨機（往上）
        mode = 3: shift_y 在 [60, 90] 之間隨機（往下）
    - 如果有明確給 shift_x / shift_y 而且 mode 是 None，就用給定的值
    """

    # 如果指定了 mode，就用 mode 來決定隨機位移
    if mode is not None:
        if mode == 0:
            shift_x = random.randint(-90, -60)
            shift_y = 0
        elif mode == 1:
            shift_x = random.randint(60, 90)
            shift_y = 0
        elif mode == 2:
            shift_y = random.randint(-90, -60)
            shift_x = 0
        elif mode == 3:
            shift_y = random.randint(60, 90)
            shift_x = 0
        # 其他 mode 值就保持原本 shift_x, shift_y

    # 先縮小
    scaled = scale_image(img, scale_percent=80)

    # 水平循環位移 (axis=1)
    translated_img = np.roll(scaled, shift_x, axis=1)

    # 垂直循環位移 (axis=0)
    if shift_y != 0:
        translated_img = np.roll(translated_img, shift_y, axis=0)

    return translated_img


In [ ]:
from tqdm import tqdm

def batch_process_images(input_dir, output_base_dir, transformations):
    """
    批次處理圖片
    
    參數:
        input_dir: 輸入圖片目錄
        output_base_dir: 輸出基礎目錄
        transformations: 變形操作字典
    """
    # 確保輸入目錄存在
    if not os.path.exists(input_dir):
        print(f"錯誤：輸入目錄不存在: {input_dir}")
        return
    
    # 創建輸出基礎目錄
    os.makedirs(output_base_dir, exist_ok=True)
    
    # 為每個變形操作創建資料集目錄
    dataset_dirs = {}
    for dataset_name in transformations.keys():
        dataset_path = os.path.join(output_base_dir, dataset_name)
        os.makedirs(dataset_path, exist_ok=True)
        dataset_dirs[dataset_name] = dataset_path
        print(f"創建資料集目錄: {dataset_path}")
    
    # 獲取所有圖片文件
    image_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif']
    image_files = []
    
    for file in os.listdir(input_dir):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            image_files.append(file)
    
    print(f"\n找到 {len(image_files)} 張圖片")
    print(f"將生成 {len(image_files) * len(transformations)} 張變形圖片\n")
    
    # 統計資訊
    stats = {name: {'success': 0, 'failed': 0} for name in transformations.keys()}
    
    # 處理每張圖片
    for img_file in tqdm(image_files, desc="處理圖片"):
        img_path = os.path.join(input_dir, img_file)
        
        # 讀取圖片
        img = cv2.imread(img_path)
        if img is None:
            print(f"警告：無法讀取圖片 {img_file}")
            continue
        
        # 獲取文件名
        file_name_without_ext = os.path.splitext(img_file)[0]
        file_ext = os.path.splitext(img_file)[1]
        
        # 執行變形操作
        for dataset_name, transform_info in transformations.items():
            try:
                transform_func = transform_info['function']
                transform_params = transform_info.get('params', {})
                
                # 執行變形
                transformed_img = transform_func(img, **transform_params)
                
                # 保存圖片
                output_filename = f"{file_name_without_ext}{file_ext}"
                output_path = os.path.join(dataset_dirs[dataset_name], output_filename)
                cv2.imwrite(output_path, transformed_img)
                
                stats[dataset_name]['success'] += 1
                
            except Exception as e:
                print(f"錯誤：處理 {img_file} 的 {dataset_name} 變形時發生錯誤: {str(e)}")
                stats[dataset_name]['failed'] += 1
    
    return stats

In [ ]:
# 定義變形操作設定
transformations = {
    # 1. 平移
    'translate_left_rand': {
        'function': translate_image_wrap,
        'params': {'mode': 0}
    },
    'translate_right_rand': {
        'function': translate_image_wrap,
        'params': {'mode': 1}
    },
    'translate_up_rand': {
        'function': translate_image_wrap,
        'params': {'mode': 2}
    },
    'translate_down_rand': {
        'function': translate_image_wrap,
        'params': {'mode': 3}
    },

    
    # 2. 旋轉（先縮小 80% 再旋轉30度以內）
    'rotate_pos_rand_30': {
        'function': scale_then_rotate_random,
        'params': {'mode': 0, 'max_angle': 30, 'scale_percent': 80}
    },
    'rotate_neg_rand_30': {
        'function': scale_then_rotate_random,
        'params': {'mode': 1, 'max_angle': 30, 'scale_percent': 80}
    },


    
    # 3. 縮放
    'scale_random_big': {
        'function': scale_image,
        'params': {'mode': 1}
    },
    'scale_random_small': {
        'function': scale_image,
        'params': {'mode': 0}
    },


    # 4. 比例改變 - 上半部分拉伸
    'resize_upper_1.5': {
        'function': resize_half_image,
        'params': {'scale_factor': 1.5, 'resize_upper': True}
    },
    
    # 5. 比例改變 - 下半部分壓縮
    'resize_lower_0.7': {
        'function': resize_half_image,
        'params': {'scale_factor': 0.7, 'resize_upper': False}
    }
}

# 設定輸入與輸出目錄
input_dir = ""
output_dir = ""

# 執行批次處理 (如果 input_dir 存在)
if os.path.exists(input_dir):
    print(f"開始處理目錄: {input_dir}")
    stats = batch_process_images(input_dir, output_dir, transformations)
    
    # 顯示結果
    print("\n處理統計:")
    for name, stat in stats.items():
        print(f"{name}: 成功 {stat['success']}, 失敗 {stat['failed']}")
else:
    print(f"找不到輸入目錄: {input_dir}，請修改 input_dir 變數")

## Part 2

Below are additional transformation to bend skeletons by swirl effect.

Change input_dir & output_dir

In [ ]:
def swirl_effect(img, angle=180, strength=1.0, center_x_ratio=0.4, center_y_ratio=0.5):
    """
    旋轉扭曲效果：只在一定半徑內做 swirl，外圈保持原圖，避免邊界變成條紋。

    參數:
        img: 輸入圖片 (BGR格式)
        angle: 中心的最大旋轉角度（度數）
        strength: 扭曲強度，控制影響範圍 (0.0-2.0)
        center_x_ratio: 旋轉中心 X 位置比例 (0.0-1.0)
        center_y_ratio: 旋轉中心 Y 位置比例 (0.0-1.0)
    """
    height, width = img.shape[:2]

    # 旋轉中心
    center_x = width * center_x_ratio
    center_y = height * center_y_ratio

    # 最大半徑（影響範圍）
    max_radius = min(width, height) * strength / 2.0
    if max_radius <= 0:
        return img.copy()

    # 座標網格
    y, x = np.indices((height, width), dtype=np.float32)

    dx = x - center_x
    dy = y - center_y
    r = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)

    # 只在 r <= max_radius 的地方做 swirl
    mask = r <= max_radius

    # 正規化半徑 (0~1)
    r_normalized = np.zeros_like(r, dtype=np.float32)
    r_normalized[mask] = r[mask] / max_radius

    # 旋轉角度（距離越遠旋轉越小）
    rotation_angle = np.zeros_like(r, dtype=np.float32)
    rotation_angle[mask] = angle * (1.0 - r_normalized[mask])

    rotation_rad = np.deg2rad(rotation_angle)
    new_theta = theta + rotation_rad

    # 先預設為「不變形」
    x_new = x.copy()
    y_new = y.copy()

    # 只有在 mask 範圍內才改座標
    x_new[mask] = center_x + r[mask] * np.cos(new_theta[mask])
    y_new[mask] = center_y + r[mask] * np.sin(new_theta[mask])

    # 避免極少數浮點誤差跑出邊界
    x_new = np.clip(x_new, 0, width - 1)
    y_new = np.clip(y_new, 0, height - 1)

    map_x = x_new.astype(np.float32)
    map_y = y_new.astype(np.float32)

    # 用 REFLECT 當邊界模式，比 CONSTANT 好看很多
    result = cv2.remap(
        img,
        map_x,
        map_y,
        cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT
    )

    return result


In [ ]:
# 定義變形操作設定
transformations = {
    'swirl_20': {
        'function': swirl_effect,
        'params': {'angle': 20, 'strength': 0.5}
    },
    'swirl_40': {
        'function': swirl_effect,
        'params': {'angle': 40, 'strength': 0.5}
    },
    'swirl_60': {
        'function': swirl_effect,
        'params': {'angle': 60, 'strength': 0.5}
    }
}

# 設定輸入與輸出目錄
input_dir = ""
output_dir = ""

# 執行批次處理 (如果 input_dir 存在)
if os.path.exists(input_dir):
    print(f"開始處理目錄: {input_dir}")
    stats = batch_process_images(input_dir, output_dir, transformations)
    
    # 顯示結果
    print("\n處理統計:")
    for name, stat in stats.items():
        print(f"{name}: 成功 {stat['success']}, 失敗 {stat['failed']}")
else:
    print(f"找不到輸入目錄: {input_dir}，請修改 input_dir 變數")